<a href="https://colab.research.google.com/github/CiaraAOC/frictionless/blob/main/frictionless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install frictionless 
import frictionless
!pip install pyjstat
import pyjstat
from pyjstat import pyjstat
import pandas as pd
from pprint import pprint

     |████████████████████████████████| 233 kB 5.0 MB/s 
     |████████████████████████████████| 636 kB 52.1 MB/s 
     |████████████████████████████████| 245 kB 45.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 45 kB 3.1 MB/s 
  Created wheel for petl: filename=petl-1.7.4-py3-none-any.whl size=216998 sha256=c41b46e900a04fe2000f32ae5c256ab22495d25065fc80266bd662e5bcde9294
  Stored in directory: /root/.cache/pip/wheels/75/23/fb/a0e8e350a8718579b0cde55163df77b94de835d7d116388fa5
  Created wheel for simpleeval: filename=simpleeval-0.9.10-py3-none-any.whl size=13769 sha256=f50f97cfa39619c74bc17bf49397319ed685abdc7ec3a4f9ed005c4098b9849a
  Stored in directory: /root/.cache/pip/wheels/d4/aa/50/0b420d1eabad3c16a82368935c6a2050955bc3ee2a11ee4e06
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3586 sha256=cf939522cd0962d57382268f1bde37a7e

In [ ]:
from pyjstat import pyjstat
dataset = pyjstat.Dataset.read('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/THA18/JSON-stat/2.0/en')

In [ ]:
df = dataset.write('dataframe')
print(df)

                            Statistic  Year  ... Year of Registration      value
0                  Vehicle Population  2015  ...            All years  1992819.0
1                  Vehicle Population  2015  ...       1999 or before    90989.0
2                  Vehicle Population  2015  ...                 2000    77870.0
3                  Vehicle Population  2015  ...                 2001    70838.0
4                  Vehicle Population  2015  ...                 2002    86226.0
...                               ...   ...  ...                  ...        ...
1140475  Average Kilometres Travelled  2019  ...                 2015    15169.0
1140476  Average Kilometres Travelled  2019  ...                 2016    13008.0
1140477  Average Kilometres Travelled  2019  ...                 2017    15019.0
1140478  Average Kilometres Travelled  2019  ...                 2018    13059.0
1140479  Average Kilometres Travelled  2019  ...                 2019    13126.0

[1140480 rows x 8 columns]


In [ ]:
from frictionless import describe #generates metadata describing layout/contents of data

resource = describe(df)
pprint(resource)

{'data':                             Statistic  Year  ... Year of Registration      value
0                  Vehicle Population  2015  ...            All years  1992819.0
1                  Vehicle Population  2015  ...       1999 or before    90989.0
2                  Vehicle Population  2015  ...                 2000    77870.0
3                  Vehicle Population  2015  ...                 2001    70838.0
4                  Vehicle Population  2015  ...                 2002    86226.0
...                               ...   ...  ...                  ...        ...
1140475  Average Kilometres Travelled  2019  ...                 2015    15169.0
1140476  Average Kilometres Travelled  2019  ...                 2016    13008.0
1140477  Average Kilometres Travelled  2019  ...                 2017    15019.0
1140478  Average Kilometres Travelled  2019  ...                 2018    13059.0
1140479  Average Kilometres Travelled  2019  ...                 2019    13126.0

[1140480 rows x 8 

In [ ]:
from frictionless import extract #read and normalise data

rows = extract(df)
#pprint(rows)

In [ ]:
from frictionless import validate #detects errors in the file

report = validate(df)
pprint(report.task.scope)

['hash-count-error',
 'byte-count-error',
 'field-count-error',
 'row-count-error',
 'blank-header',
 'extra-label',
 'missing-label',
 'blank-label',
 'duplicate-label',
 'incorrect-label',
 'blank-row',
 'primary-key-error',
 'foreign-key-error',
 'extra-cell',
 'missing-cell',
 'type-error',
 'constraint-error',
 'unique-error']


In [ ]:
from frictionless import Resource, FrictionlessException

try:
    resource = Resource(df)
except FrictionlessException as exception:
    pprint(exception.error)
    # Prints the SchemaError metadata in this case

In [ ]:
from pprint import pprint
from frictionless import validate, checks

checks = [checks.sequential_value(field_name='Statistic')]
report = validate(df, checks=checks)
pprint(report.flatten(["rowPosition", "fieldPosition", "code", "note"]))

[[2, 1, 'sequential-value', 'the value is not sequential'],
 [None, None, 'task-error', 'exceeded memory limit "1000MB"']]


In [ ]:
from frictionless import Resource, transform, steps

# Define source resource
source = Resource(name='main', path=df)
# Apply transform steps
target = transform(
    source,
    steps=[
        steps.table_normalize()
    ],
)

# Print resulting schema and data
pprint(target.schema)
print(target)
print(target.data)
pprint(source.read_lists())
pprint(target.resource_names)


In [ ]:
def clean(resource):
  current = resource.to_copy

source = describe(df)
target = transform(
    source,
    steps=[
        clean,
        steps.table_write(path="THA18.xlsx"),
    ],
)

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.table_normalize(),
        steps.table_pivot(f1="Statistic", f2="Year", f3="Type of Ownership", aggfun=sum),
    ]
)
pprint(target.schema)
pprint(target.read_rows())
pprint(target)

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.table_normalize(),
        steps.table_transpose(),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
   steps=[
        steps.cell_set(field_name="Statistic", value="bad"),
        steps.table_validate(),
    ]
)
pprint(target.schema)
try:
  pprint(target.read_rows())
except Exception as exception:
  pprint(exception)

In [ ]:
source = Resource(path=df)
target = transform(
    source,
    steps=[
      steps.table_debug(function=print),
    ],
)
pprint(target)

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Package(resources=[Resource(name='main', path=df)])
target = transform(
    source,
    steps=[
        steps.resource_add(name='extra', path=df),
    ],
)
pprint(target.resource_names)
pprint(target.get_resource('extra').schema)
pprint(target.get_resource('extra').read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Package(resources=[Resource(name='main', path=df)])
target = transform(
    source,
    steps=[
        steps.resource_add(name='extra', path=df),
        steps.resource_transform(name='main', steps=[
            steps.table_merge(resource='extra'),
            steps.row_sort(field_names=['Statistic'])
        ]),
        steps.resource_remove(name="extra"),
    ],
)
pprint(target.resource_names)
pprint(target.get_resource('main').schema)
pprint(target.get_resource('main').read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.field_filter(names=["Year", "County of Ownership"]),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.field_move(name="Statistic", position=2),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.field_split(name="Type of Ownership", to_names=["Male", "Female"], pattern="a"),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

In [ ]:
from frictionless import Parser

class HtmlParser(Parser):
    requires_loader = True
    supported_types = [
        "string",
    ]

    # Read

    def read_list_stream_create(self):
        pq = helpers.import_from_plugin("pyquery", plugin="html").PyQuery
        dialect = self.resource.dialect

        # Get Page content
        page = pq(self.loader.text_stream.read(), parser="html")

        # Find required table
        if dialect.selector:
            table = pq(page.find(dialect.selector)[0])
        else:
            table = page

        # Stream headers
        data = (
            table.children("thead").children("tr")
            + table.children("thead")
            + table.children("tr")
            + table.children("tbody").children("tr")
        )
        data = [pq(r) for r in data if len(r) > 0]
        first_row = data.pop(0)
        headers = [pq(th).text() for th in first_row.find("th,td")]
        yield headers

        # Stream data
        data = [pq(tr).find("td") for tr in data]
        data = [[pq(td).text() for td in tr] for tr in data if len(tr) > 0]
        yield from data

    # Write

    def write_row_stream(self, resource):
        source = resource
        target = self.resource
        html = "<html><body><table>\n"
        with source:
            for row in source.row_stream:
                if row.row_number == 1:
                    html += "<tr>"
                    for name in row.field_names:
                        html += f"<td>{name}</td>"
                    html += "</tr>\n"
                cells = row.to_list(types=self.supported_types)
                html += "<tr>"
                for cell in cells:
                    html += f"<td>{cell}</td>"
                html += "</tr>\n"
        html += "</table></body></html>"
        with tempfile.NamedTemporaryFile("wt", delete=False) as file:
            file.write(html)
        loader = system.create_loader(target)
        result = loader.write_byte_stream(file.name)
        return result
        pprint(target)

In [ ]:
from frictionless import Parser

class HtmlParser(Parser):
    requires_loader = True
    supported_types = [
        "string",
    ]

    # Read

    def read_list_stream_create(self):
        pq = helpers.import_from_plugin("pyquery", plugin="html").PyQuery
        dialect = self.resource.dialect

        # Get Page content
        page = pq(self.loader.text_stream.read(), parser="html")

        # Find required table
        if dialect.selector:
            table = pq(page.find(dialect.selector)[0])
        else:
            table = page

        # Stream headers
        data = (
            table.children("thead").children("tr")
            + table.children("thead")
            + table.children("tr")
            + table.children("tbody").children("tr")
        )
        data = [pq(r) for r in data if len(r) > 0]
        first_row = data.pop(0)
        headers = [pq(th).text() for th in first_row.find("th,td")]
        yield headers

        # Stream data
        data = [pq(tr).find("td") for tr in data]
        data = [[pq(td).text() for td in tr] for tr in data if len(tr) > 0]
        yield from data

    # Write

    def write_row_stream(self, resource):
        source = resource
        target = self.resource
        html = "<html><body><table>\n"
        with source:
            for row in source.row_stream:
                if row.row_number == 1:
                    html += "<tr>"
                    for name in row.field_names:
                        html += f"<td>{name}</td>"
                    html += "</tr>\n"
                cells = row.to_list(types=self.supported_types)
                html += "<tr>"
                for cell in cells:
                    html += f"<td>{cell}</td>"
                html += "</tr>\n"
        html += "</table></body></html>"
        with tempfile.NamedTemporaryFile("wt", delete=False) as file:
            file.write(html)
        loader = system.create_loader(target)
        result = loader.write_byte_stream(file.name)
        return result
        pprint(target)

In [ ]:
import os.path
from frictionless import Resource
resource = Resource(df)
from frictionless.plugins.excel import ExcelDialect
resource.write("File.xlsx", dialect=ExcelDialect(sheet='My Table'))